# Assignment 09

**Due:** 2022-05-13 (Friday), 11:59 PM, as a Jupyter notebook (with related files) submitted via your repo in the course GitHub organization.  Use the solutions template notebook, and **be sure not to modify the subproblem cells** in your solution file.  This assignment is **mandatory**.

**Extensions:** Note that the deadline is the day before final exams commence. In the past, Cornell has forbidden instructors from extending deadlines into the finals period. The rule may no longer apply, but please consider the deadline to be firm unless truly unusual circumstances arise.

**NOTE:** Doing all of the calculations for these problems entirely in your Solutions Jupyter notebook will make the notebook unwieldy (both to work on, and to grade).  In addition, you may find it easier to debug or iterate on your analysis in a script rather than in a notebook.

Accordingly, aim to *include only requested results in your Solutions notebook* for this assignment.  Some of the request results are *summaries* of work that should be done in a separate script or supplementary notebook—particularly in regard to MCMC output analysis (e.g., checking for sufficient burn-in and ESS). In this regard, what should be in your Solutions notebook should resemble what you would include in a published paper or report (supported by additional calculations not fully presented in the publication), except that your Solutions notebook will include runnable code.

Your companion script or notebook may use separate runs of your models. In a publication setting, you would use those scripts to produce plots to include in your publication. Here, since your "publication" (the Solutions notebook) will be runable, you won't be describing diagnostics done for the exact runs we see in your Solutions, but the behavior should be similar for multiple runs of adequate length.

**Make sure you commit, not just your Solutions notebook, but also any Python module, scripts, supplementary notebook, or images needed to run the notebook and to produce the content included or described in it.**

**Do not commit your cached Stan model files.** Those are platform-dependent. Your `.gitignore` should automatically exclude them.

### Collaboration disclosure

(You may leave this cell unmodified if you worked entirely alone on this assignment.)

## Accompanying scripts or notebooks:

myscript.py **<-- EDIT if you have uploaded accompanying scripts**

In [1]:
# The usual imports:
import numpy as np
import scipy
import matplotlib as mpl
from matplotlib.pyplot import *
from numpy import *
from scipy import stats

from stanfitter import StanFitter

## 1. Hierarchical Bayes and cancer cluster small area estimation

**Note:  This exercise does not comprise a serious analysis of cancer cluster data!**  This exercise is meant to give you some experience with hierarchical Bayesian modeling with Stan, and to help you gain some insight into issues that arise in small area estimation.

A Python module, implemented in "cancer_data.py", accompanies this assignment; it includes data you'll use for your solutions.  Also, you'll use the version of "stanfitter.py" that accompanies this assignment (it's modified vs. the version used for A08).  **Make sure you commit, not just your notebooks, but also all of the Python modules and scripts needed to run the notebooks and to produce the content included in them.**

## Small area estimation

[Small area estimation](http://en.wikipedia.org/wiki/Small_area_estimation) (SAE) is an area of statistics that deals with problems involving estimating properties of **sub**-populations, viewed in the context of a much larger population.  A protypical problem in small area estimation arises in epidemiology and the study of disease or illness clusters: is the prevalence of a condition significantly greater in a particular sub-population than it is across the broad population? And if so, can the cause of increased prevalence be identified?  Survey data, providing incomplete but hopefully representative samples of the full population and sub-populations, play a key role in SAE.

Many statistical issues arise in SAE problems.  Two prominent ones that will concern us in this assignment are:

* **Small sample size statistics:**  Sub-population sample sizes are typically small compared to samples from the full population.  Even when a sub-population is completely sampled, the condition being studied may be rare, requiring estimation in regimes where asymptotic approximations (e.g., using a normal approximation for binomial or Poisson data) are not justified.  The ability of Bayesian methods to work without approximation with small datasets has made Bayesian statistics increasingly important in SAE.

* **Multiplicity issues:**  When there are *many* possible sub-populations (typically the case if the sub-populations are small), there are many possibilities for a rare outcome to occur simply by chance, due to the multiplicity of sub-populations.  For example, an outcome that one would expect to arise only 0.1% of the time under "normal" circumstances is likely to appear once in 1000 independent observations.  As a result, a particular sub-population may appear to have a significantly anomalous prevalence for a rare condition, even if there is no cause for increased prevalence, if it is one among many examined sub-populations.  Proper assessment of SAE results needs to account for the multiplicity of sub-populations, i.e., that a sub-population is itself a member of a population (of sub-populations).

This exercise has you use Bayesian methods to address the first issue directly, and the second issue indirectly, through hierarchical Bayesian (multilevel) modeling.  Especially in regard to multiplicity, it represents only starting points for more thorough analyses.

## The Niles leukemia cancer cluster

A famous SAE problem arose in Cook County, Illinois, the county containing the city of Chicago.  In the small village of Niles in early 1961, 4 children in the same neighborhood were diagnosed with leukemia within a short period of time of each other.  A subsequent look at records for the village found 4 more cases around the same time.  Based on childhood leukemia counts across the county, the number of cases in Niles appeared anomalously high.  For Problem 4.1, you'll look at part of the evidence relevant to assessing childhood leukemia prevalence in Niles at the time.

For a brief retrospective on the case (with a nontechnical discussion of some of the statistical issues), see ["In Final Analysis, 'Cancer Clusters' Often Just Chance"](https://www.chicagotribune.com/news/ct-xpm-1994-06-07-9406070122-story.html).

Here are the basic numbers, for the village, and for Cook County as a whole, regarding childhood leukemia cases for a 5-yr period ending about the time of the Niles cases:

In [2]:
# Cook county data:
nl_c = 286  # number of children with leukemia in Cook county
nc_c = 1152695  # number of children in Cook county

# Village of Niles data:
nl_n = 8  # number of children with leukemia in Niles
nc_n = 7076  # number of children in Niles

The most fundamental way to model such data—counts of a binary outcome in a fixed number of trials—is via the *binomial distribution*.  The natural parameter would be the probability, $\alpha$, for a particular (randomly chosen) child to be diagnosed with leukemia in a 5 year period.  A Bayesian analysis is certainly feasible with such a setup.

However, the outcome is exceptionally rare, and in the setting of a large total sample size, $N$ (total number of children), with a rare outcome, a very good approximation to the binomial is the **Poisson distribution**, with the parameter being $\mu = \alpha N$, the expectation value for the number of rare outcomes in a population of size $N$.  This is the standard parameterization used in epidemiology.  One typically defines $\mu$ in reference to a specific, convenient "fiducial" population size and period of observation.  Here, we will take $\mu$ to be defined as the **expected number of cases in a population of one million children observed for a year**, an easy-to-interpret quantity.

With this definition, for a population of size $N$ observed for $y$ years, the expected number of cases is
$$
\nu = \mu y \frac{N}{10^6}.
$$
We'll sometimes refer to the multiplier, $\epsilon \equiv y N/10^6$, as the **exposure** for the observations of the population.

With these definitions, the probability distribution for the number of cancer cases, $n$, seen in a population of size $N$ monitored for $y$ years is Poisson with expectation value $\nu$:
$$
n \sim {\rm Poisson}(\mu y N/10^6).
$$

As a simple way to explore a possible change in cancer rate in a sub-population, consider introducing a nonnegative "multiplying factor" parameter, $f_i$, defined so that the actual rate in sub-population $i$, with $N_i$ total members observed over $y_i$ years, is
$$
\nu_i = f_i \mu y_i \frac{N_i}{10^6},
$$
with the number of cases in that population having a Poisson distribution with expectation value $\nu_i$:
$$
n_i \sim {\rm Poisson}(f_i \mu y_i N_i/10^6).
$$

Although we're approximating the binomial with a Poisson distribution here, the approximation is very good.  However, for small counts data like the Niles data, further approximating the Poisson (or the binomial) with a Gaussian distribution is likely to be inaccurate.

(For simplicity, throughout this problem we'll ignore the fact that the Niles numbers are themselves presumably included in the Cook County counts.  The relatively small size of the Niles numbers indicates we should be safe doing so.)

### Problem 1.1 (2 points):

**Quick-and-dirty estimation:** From the data above and what you know about inference with the Poisson distribution with *large* counts (e.g., the "root-$n$" rule), consider Niles and Cook County to have *separate* cancer rates, $\mu_n$ and $\mu_c$, and compute point estimates with uncertainties specified as approximate posterior standard deviations.  You needn't present a detailed derivation; just compute the numbers in the notebook, and print them.  Include computation of the exposures for the Niles and Cook County data.  Also print the *ratio* of the estimated Niles rate to the estimated Cook county rate (you needn't formally propagate uncertainty for the ratio).  Use these results to *briefly* explain why Niles residents were initially worried.

### Problem 1.2 (5 points):

**Formal inference with the Poisson distribution:**  Now model the data more accurately, using a model where the Cook County data are described by a base cancer rate $\mu$, and the Niles data are modeled with a rate $f\mu$, with $f \ge 0$.

 * Write a Stan model for this setup (complete the stub below).  Take the prior for the Cook County rate to be exponential with an expectation value of $1000$, i.e., reflecting that we expect cancer rates to be below 1000 cases per million people per year (but still allowing for high values).  Take the prior for $f$ to be a gamma distribution with shape parameter $\alpha=1.5$, and with a mean of 1 (i.e., a broad prior, but with the expected $f=1$).  Make sure you understand Stan's parameterizations when you set the priors.  
 * Use StanFitter to fit the model to the four numbers provided above.  Summarize your analysis (including diagnostic work) textually in the notebook (a short paragraph or two should suffice).  Include a run of the fit in your Solutions notebook.  Use that run to include two important results:
   * A plot of the marginal posterior for $f$ (for grading purposes, this is the only plot *required* to be shown in the notebook). You may produce this via the StanFitter `stan_plot()` method, `ArviZ`, or by making a plot of your own design using `pyplot` or another Python tool.
   * Using the posterior samples for $f$, compute the posterior probability that $f>1$, i.e., that the rate in Niles is greater than the overall Cook County rate.

That's enough for the assignment, but note that even for such a simple model, a thorough analysis would include work pertaining to robustness with respect to the priors.

In [3]:
leuk_code = """
data {
    int<lower=0> nl_large; // # of large-sample leukemia cases
    int<lower=0> nc_large; // # of children in large sample
    int<lower=0> nl_small; // # of small-sample leukemia cases
    int<lower=0> nc_small; // # of children in small sample
}

parameters {
    real<lower=0> mu_large;  // rate per yr per 1e6 in large sample
    real<lower=0> fac;  // factor giving rate for small sample
}

model {
    // FILL IN HERE
}
"""

# nfitter = StanFitter(leuk_code)
ndata = dict(nl_large=nl_c, nc_large=nc_c, nl_small=nl_n, nc_small=nc_n)

## Multilevel modeling of cancer cluster data

You should find a fairly large probability that the Niles rate, *considered in isolation*, is larger than the overall Cook County rate.  However, ultimately it was determined that the Niles data were adequately explained as a chance result, due largely to the *multiplicity* considerations mentioned above.

There are several approaches for accounting for multiplicity in small area estimation.  From the Bayesian point of view, multilevel or hierarchical modeling provides a flexible framework for describing multiplicity effects. We won't explore this further for the Niles data here, but we'll consider a related problem whose analysis will take us a step up in complexity.

For Problem 2, you'll develop a simple multilevel model for data resembling the Niles data, but considering several small sub-populations that together comprise a larger sub-population, situated in an even larger general population.  To make the problem interesting, we'll look at data rather close to home:  the general population will be _**New York state**_, the intermediate population will be _**Tompkins County**_, and among the subpopulations will be _**Ithaca and surrounding areas**_.

We'll model a small chunk of data taken from the the [New York State Dept. of Health](http://www.health.ny.gov/) *New York State Cancer Registry*.  I've collected the data for you, but you may want to learn more about it from these resources:

* [About the New York State Cancer Registry](http://www.health.ny.gov/statistics/cancer/registry/about.htm)
* [About the Data](https://www.health.ny.gov/statistics/cancer/environmental_facilities/mapping/about/)
* [Cancer Incidence by ZIP Code](https://web.archive.org/web/20200709155506/http://www.health.ny.gov/statistics/cancer/registry/zipcode/index.htm) (archived)

The first two links describe the latest version of data in the registry, covering 2010-2014. Those data are for census tracts, very small neighborhood-based subdivisions (typically containing about 4000 people). We will instead look at slightly older data reporting cancer cases by ZIP code (typically representing cities, towns, and villages). If you'd like to explore the more recent data, you'll find it here: [Cancer Incidence by Census Tract](https://www.health.ny.gov/statistics/cancer/registry/tract/index.htm).

The Registry's ZIP code data reports cancer case counts, and expected counts, for four different types of cancer, over the 5-yr period 2005-2009.  We'll consider two:  *breast cancer in women*, and *prostate cancer* (in men).  I've provided the data in the accompanying `cancer_data` module.  FYI: I gathered it by using the handy [ConvertCSV](http://www.convertcsv.com/) web site.  Although Python has solid tools for both scraping web pages for data, and for parsing and converting scraped data, ConvertCSV is easier to use for small amounts of data.

In [4]:
from cancer_data import bc_counts, bc_expect, pc_counts, pc_expect, ith_indx

Here `bc_counts` is an array of breast cancer case counts (integers) for 11 primary zip code areas in Tompkins County, and `bc_expect` is an array of expectation values for counts (real-valued) for the same list of areas.  Likewise, `pc_counts` and `pc_expect` provides data for prostate cancer.  The integer, `ith_indx`, is the index for Ithaca (primary zip 14850); its value is 6.  See the tables in the last URL listed above for details about the areas included.

The expected values reported in the tables are computed in a nontrivial manner.  They are based on state-wide data correlating cancer rates with age and race.  For each area, the composition of the population is used to compute the expected rate, accounting for the distribution of ages and races in the area.  For our purposes, consider these predicted rates to be precisely known.

As a simple model for these data, we'll consider the rate in area $i$ to be a factor $f_i$ times the predicted rate, with the $f_i$ related to each other in the sense of being drawn from a common distribution. I.e., we are allowing for the possibility that the predictions based on the state-wide analysis are systematically off in our region.  If the distribution for the $f_i$ values is centered near $1$, or away from $1$ but broad, with high density at $f_i=1$, then there is little evidence of anomalous risk (either low or high) in Tompkins county.  But if the $f_i$ distribution has most of its probability away from $f_i=1$ (e.g., it's a narrow distribution with a peak away from $1$), then there is evidence of anomalous risk in the county.

Note that this model permits variability within our region.  In fact, if rates are generically quite variable, this would show up as a broad $f_i$ population distribution, in which case we'd discover that it isn't surprising for some areas to appear to have anomalous rates.

Specifically, adopt a model with Poisson distributions for the counts, where the expectation value for the counts in area $i$ is
$$
\nu_i = f_i \tilde{\nu}_i,
$$
where $\tilde{\nu}_i$ is the predicted expected counts (based on state-wide data) for area $i$, as reported in the `bc_expect` and `pc_expect` arrays.  Take the $f_i$ population distribution to be a gamma distribution,
$$
f_i \sim {\rm Gamma}(\alpha,\beta),
$$
where $\alpha$ is the shape parameter and $\beta$ the inverse scale parameter (or "rate parameter" in the terminology of Wikipedia's [Gamma distribution](https://en.wikipedia.org/wiki/Gamma_distribution) entry).  But for the actual parameters you'll track with Stan, use the mean, $\mu_f$, and standard deviation, $\sigma_f$, of the population gamma distribution. You should look up (or derive!) the relationships between $(\alpha,\beta)$ and the gamma mean and standard deviation (but the Stan code provided below already uses the relationships).

Take the prior for the mean to be a relatively uninformative gamma distribution,
$$
\mu_f \sim {\rm Gamma}(\alpha=1.5,\beta=1.5).
$$
This vanishes at 0 (since we don't expect the $f_i$ values to vanish), but otherwise is broad, with a mean at 1 and a standard deviation of $\approx 0.82$ (the central 90% region of this prior spans from $f\approx 0.1$ to $f\approx 2.6$, so it readily accomodates pretty large departures from the predictions of the state-wide model).

Take the prior for the standard deviation to be another gamma distribution,
$$
\sigma_f \sim {\rm Gamma}(\alpha=1.,\beta=1.);
$$
this allows vanishingly small $\sigma_f$ (corresponding to the the state-wide predictions all being correct), but otherwise is noncommital over regions of $\sigma_f$ up to a few.

### Problem 2.1 (3 points):

> * Draw the DAG corresponding to this model.  Use any tool you wish to draw it; only include the final figure in your notebook (with a description of what tool you used if it's not obvious).  Most diagramming or drawing applications will be good enough for this simple DAG.  You might consider learning about one of the following (see the lab notebook on drawing DAGs for coverage of some of these):
>   * [graphviz](http://www.graphviz.org/):  This is an "industry standard" for graph drawing.  It is based on a simple language called *dot* for describing graphs via lists of node names and edge descriptions.  *graphviz* itself is a collection of command-line tools that process *dot* files and produce graphical output.  Several other graph drawing tools use the *dot* language.  Mac users can easily install *graphviz* via Homebrew; it is widely available in Linux package managers.  There are multiple Python packages providing Python interfaces to *graphviz* ([PyPI search](https://pypi.python.org/pypi?%3Aaction=search&term=graphviz&submit=search)).
>   * [DAFT](https://docs.daft-pgm.org/en/latest/):  This is a Python package that uses matplotlib to draw nice-looking graphs.
>   * I use [OmniGraffle](https://www.omnigroup.com/omnigraffle) for the graphs shown in lectures; it's a commercial diagramming app (Mac and iOS only) that's not cheap, but worth it (there is an academic discount). [Microsoft Visio](https://www.microsoft.com/en/microsoft-365/visio/flowchart-software) is a Windows tool offering similar capability. Unlike OmniGraffle, it does not have built in support for the *graphviz* *dot* language, but the [GraphVizio](https://www.calvert.ch/graphvizio/) Visio add-in provides some *dot* support.
>   * Presentation software like Apple's Keynote or Microsoft's PowerPoint can be used to draw simple graphs, and could work for this problem.
>   * Feel free to draw the DAG by hand and include a scan or photo in your notebook (use a compressed image format supported by the notebook, like PNG or JPG; please, no selfies!).

> * Complete the Stan model stub below to model the data as described above.  Note that it includes a `transformed parameters` block relating the $\mu_f$ and $\sigma_f$ parameters to the $(\alpha,\beta)$ that Stan needs for the population gamma distribution.  Read the Stan manual if you'd like to learn more about this capability (see [Stan Language Reference Manual - Section 8, Program Blocks](https://mc-stan.org/docs/2_23/reference-manual/blocks-chapter.html)).  Compile the model with StanFitter in the notebook.

In [5]:
# NY state cancer registry model:
cancer_code = """
data {
    int<lower=0> n_a; // # of areas
    int counts[n_a]; // cancer cases in each area
    real expect[n_a]; // expected # of cases in each area
}

parameters {
    real<lower=0.> fac[n_a];  // factor = actual prevalence/expected
    real<lower=0.> mu_f;  // pop'n mean for factors
    real<lower=0.> sigma_f;  // pop'n std dev'n for factors
}

transformed parameters {
    real<lower=0.> alpha;  // gamma dist'n alpha
    real<lower=0.> beta;  // gamma dist'n beta
    alpha = mu_f*mu_f / (sigma_f*sigma_f);
    beta = mu_f / (sigma_f*sigma_f);
}

model {
    FILL IN HERE
}
"""

# cfitter = StanFitter(cancer_code)

bc_data = dict(n_a=len(bc_counts), counts=bc_counts, expect=bc_expect)
pc_data = dict(n_a=len(pc_counts), counts=pc_counts, expect=pc_expect)

### Problem 2.2 (4 points):

> * Write a function with signature `fit_plots(fit)` that takes a `StanFitResults` instance (from running the model's `sample()` method) and makes the plots described in the two bullets below. Include the function in your Solutions notebook, or import it from a companion Python module so you can run it in the notebook.  Arrange the plots however you see fit, but try to consolidate the results while keeping them easy to understand.  For example, my first try used a single `figure` with 5 subplots; the $f_i$ marginals were shown by plotting 3 per subplot (2 in the last), in 4 of the subplots, as translucent histograms (I found showing more than 3 per subplot made them too hard to distinguish), and the scatterplot was in the last subplot. That is just one way to do it.
>    * Marginal distributions for all 11 $f_i$ factors;
>    * A scatterplot displaying samples from the marginal posterior density for $(\mu_f,\sigma_f)$, with $\mu_f$ along the $x$ axis.

> * Fit the breast cancer data, and display the marginals with `fit_plots`, in the notebook.  Be sure to look at MCMC diagnostics (outside of the Solutions notebook), *briefly* recording observations in your notebook from your analysis (you needn't include diagnostic plots or many details).  In my preliminary runs, I found that to get useful effective sample sizes (at least 1000; a few thousand is better), I needed far more than the few 1000 iterations we've sometimes used to get satisfactory results with simpler models.  Briefly summarize what the plots indicate about the local breast cancer rates (of course, this is a simplified analysis, so do not take the results too seriously).

> * Similarly, fit the prostate cancer data, summarize your analysis, display the results with `fit_plots`, and briefly describe what they indicate about local rates.

**Note:** You will likely see some warning messages from Stan about the internal E-BFMI (Energy Bayesian Fraction of Missing Information) being low in some chains, or that there are divergences. This is an indication that the NUTS sampler is having trouble exploring some parts of the parameter space. This can happen here if population distributions with $\sigma_f \approx 0$ are consistent with the data, because this effectively corresponds to a $\delta$ function population distribution for the $f_i$ values. You may ignore this warning in this assignment (the values of BFMI shouldn't be *too* much lower than the 0.2 value indicating some concern), but in a more careful analysis we would consider a different population description (perhaps comparing models with $\delta$ functions to broader models). For more discussion of this (with pointers to relevant literature), see the lab exercise notebook on beta-binomial modeling of baseball batting average data.

One relatively simple way you may be able to reduce the divergences or E-BFMI warnings is to force Stan to use a smaller step size then it's default adaptive step size. You can do this by adding a `control` keyword argument when you call a fitter object's `sample()` method: `sample(..., control={'adapt_delta': 0.99})`. The `adapt_delta` defaults to 0.95; making it closer to 1 decreases the NUTS sampler step size, which will make sampling take longer. Feel free to explore this if you like, but it's not required.

At this point, your assignment is complete.  In regard to the main statistical and epidemiological issues raised in this problem, you may find the following resources interesting for future (post-finals!) reading:

* An elementary discussion of use of the Poisson distribution in epidemiology (including discussion of statistical issues in assessing disease clusters), in Ch. 19 of [Epidemiology Kept Simple](http://www.sjsu.edu/faculty/gerstman/eks/) by B. Gerstman (2013). CULib has this available as an eBook: [Epidemiology kept simple (John Wiley & Sons)](https://newcatalog.library.cornell.edu/catalog/8047662).
* Papers on Bayesian modeling and multiplicity by Jim Berger and James Scott (particularly good papers among several on the topic):
  * ["Bayes and empirical-Bayes multiplicity adjustment in the variable-selection problem"](https://projecteuclid.org/euclid.aos/1278861454)
  * ["An Exploration of Aspects of Bayesian Multiple Testing"](https://www.sciencedirect.com/science/article/abs/pii/S0378375805002156)

* Andrew Gelman's blog post and article on the meaning of statistical significance:
  * ["The statistical significance filter"](http://andrewgelman.com/2011/09/10/the-statistical-significance-filter/) (Gelman's blog has several insightful posts on interpreting statistical significance.)
  * ["The Difference Between 'Significant' and 'Not Significant' is not Itself Statistically Significant"](http://www.tandfonline.com/doi/abs/10.1198/000313006X152649#)

* "Small area mapping of prostate cancer incidence in New York State (USA) using fully Bayesian hierarchical modelling" by Glen D Johnson, published in *International Journal of Health Geographics* in 2004, available at [this URL](http://www.ij-healthgeographics.com/content/3/1/29).  This is a thorough hierarchical Bayesian analysis of the full New York state cancer registry dataset for the years 1994-1998.